In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
import requests

In [32]:
r = requests.get("https://bloxroute.max-profit.blxrbdn.com/relay/v1/data/bidtraces/proposer_payload_delivered?block_number=16712284")
r = r.json()

In [33]:
r[0]['builder_pubkey']

'0xafc9274fe595e8cff421ab9e73b031f0dff707ea1852e2233ff070ef18e3876e25c44a9831c4b5f802653d4678ccc31f'

In [18]:
relays = {
    'flashbots': "https://boost-relay.flashbots.net/",
    'aestus': "https://mainnet.aestus.live/",
    'relayooor': "https://relayooor.wtf/",
    'blox-ethical': "https://bloxroute.ethical.blxrbdn.com/",
    'blox-maxprofit': "https://bloxroute.max-profit.blxrbdn.com/",
    'blox-regulated': "https://bloxroute.regulated.blxrbdn.com/",
    'manifold': "https://mainnet-relay.securerpc.com/",
    'blocknative': "https://builder-relay-mainnet.blocknative.com/",
    'eden': "https://relay.edennetwork.io/",
    'agnostic': "https://agnostic-relay.net/",
}

In [21]:
feb26 = pd.read_csv('alchemy-data/resfeb26.txt', delimiter=' ', names=['block', 'hash', 'winning'])
feb26.head()

,block,hash,winning
0,16707626,0x1f519fb9af7a8f72b3ee60eed9d21db2afcadb537329...,1
1,16707627,0x7a031a2f3354757db9744a3a7e55c2e179168f1183b4...,1
2,16707630,0x1606e495a44b83b0309023db24fb2d49ddf2383bcb5f...,1
3,16707633,0xe3f1f445b5de9e6132c19a0cd97a31ab062b16087e1a...,1
4,16707643,0x1fd2a118b50ce5fe8e4eb7200a147511a1b4852c63b4...,1


In [23]:
slots, winning = len(feb26), len(feb26[feb26['winning']>0])
winning, slots, winning/slots

(2044, 2548, 0.8021978021978022)

In [173]:
(2044) / (2548-170)

0.8595458368376787

In [24]:
losing = feb26[feb26['winning']==0]
losing.head()

,block,hash,winning
14,16707672,0xaa6824b6376302c8f856c5b263f3017d4c244ef81147...,0
20,16707686,0xac9500af143bbb12cd1b8c41ff50cd84432de956d82b...,0
33,16707713,0xab848eb1b4678fa2aa8719826eea9b31a11cf019deba...,0
45,16707751,0x693f3b8733e86d83bec465b4f24124c3162aaffeab08...,0
46,16707752,0xc6dc2da6c859037c61014006a2b1ffee9152b2a7a317...,0


In [ ]:
f = open("missing.csv", "w")
blocknums = losing['block'].values
for bn in blocknums:
    found = False
    for r in relays:
        req = requests.get("{}relay/v1/data/bidtraces/proposer_payload_delivered?block_number={}".format(
            relays[r], bn))
        req = req.json()
        if len(req) != 0:
            pk = req[0]['builder_pubkey']
            outst = "{},{},{}\n".format(bn, r, pk)
            print(outst, end='')
            f.write(outst)
            found = True
    if not found:
        outst = "{},,\n".format(bn)
        print(outst, end='')
        f.write(outst)
f.close()

In [114]:
missing = pd.read_csv('alchemy-data/missing.csv', keep_default_na=False)
missing.head()

,blocknum,relay,pubkey
0,16707672,,
1,16707686,,
2,16707713,,
3,16707751,,
4,16707752,blox-maxprofit,0xafc9274fe595e8cff421ab9e73b031f0dff707ea1852...


In [112]:
missing = missing.groupby(['relay']).size().reset_index(name='missing_blocks')
missing = missing.sort_values('missing_blocks', ascending=False)
missing

,relay,missing_blocks
0,,170
5,blox-maxprofit,164
3,blocknative,91
4,blox-ethical,41
8,flashbots,15
7,eden,12
2,agnostic,11
9,manifold,9
1,aestus,3
6,blox-regulated,1


In [115]:
missingpks = missing.groupby(['pubkey']).size().reset_index(name='missing_blocks')
missingpks = missingpks.sort_values('missing_blocks', ascending=False)
missingpks

,pubkey,missing_blocks
0,,170
21,0xafc9274fe595e8cff421ab9e73b031f0dff707ea1852...,95
1,0x8000008a03ebae7d8ab2f66659bd719a698b2e74097d...,83
15,0x94aa4ee318f39b56547a253700917982f4b737a49fc3...,51
22,0xb086acdd8da6a11c973b4b26d8c955addbae4506c78d...,26
3,0x82801ab0556f7df1fb9bb3a61ca84beea8285a8dc3c4...,15
6,0x83d3495a2951065cf19c4d282afca0a635a39f6504bd...,15
17,0x976e63c505050e25b70b39238990c78ddf0948685eb8...,9
14,0x91970c2db7c12510acb2e9c45844f7de602f83a7f310...,9
11,0x8ed74896fb129ac7f48d6a2908400044103b65d5c4e8...,8


In [96]:
ourpks = pd.read_csv('alchemy-data/pks.csv')
ourpks.head()

,pubkey,description
0,0x8c8a7c5dc5e164d6e567bb21e3e436600618099c1c15...,lightspeed?(stale)
1,0x90abb47cfa67cb2c84ee8e1eb648206755e20c1b08ce...,unknown(stale)
2,0x8ff133b960e7b656cb5b6406c5083e0e0a7fe412b071...,emptyset(stale)
3,0xacea6ab73edee8d7c053520b4aa5d45b29692a112ae8...,riposte(stale)
4,0x8ecd091acecda7907aefa2a86253de9e4b4ba3d1d69b...,eth-builder


In [131]:
ourmissing = missingpks.merge(ourpks, on='pubkey')
ourmissing

,pubkey,missing_blocks,description
0,0x94aa4ee318f39b56547a253700917982f4b737a49fc3...,51,bloxroute
1,0x83d3495a2951065cf19c4d282afca0a635a39f6504bd...,15,rsync
2,0x976e63c505050e25b70b39238990c78ddf0948685eb8...,9,bloxroute
3,0x8ed74896fb129ac7f48d6a2908400044103b65d5c4e8...,8,eth-builder
4,0xb5d883565500910f3f10f0a2e3a031139d972117a3b6...,5,beaver
5,0x96a59d355b1f65e270b29981dd113625732539e955a1...,3,beaver
6,0x8ea1393f49d894ae22ec86e38d9aeb64b8336dac947e...,2,eth-builder
7,0x8b8edce58fafe098763e4fabdeb318d347f9238845f2...,1,bloxroute
8,0x984357f7e6489e5e04049920bf63087ee6904272e937...,1,finest(loser)
9,0x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c...,1,buildai


In [132]:
ourmissing['missing_blocks'].values.sum()

98

In [119]:
missingpks.loc[~missingpks['pubkey'].isin(ourpks['pubkey'].values)]

,pubkey,missing_blocks
0,,170
21,0xafc9274fe595e8cff421ab9e73b031f0dff707ea1852...,95
1,0x8000008a03ebae7d8ab2f66659bd719a698b2e74097d...,83
22,0xb086acdd8da6a11c973b4b26d8c955addbae4506c78d...,26
3,0x82801ab0556f7df1fb9bb3a61ca84beea8285a8dc3c4...,15
14,0x91970c2db7c12510acb2e9c45844f7de602f83a7f310...,9
25,0xb8fceec09779ff758918a849bfe8ab43cea79f6a9832...,6
8,0x8d0187110022b21a97a265d9b98aae51cdd563e53cea...,6
20,0xa25f5d5bd4f1956971bbd6e5a19e59c9b1422ca25358...,3
12,0x9000009807ed12c1f08bf4e81c6da3ba8e3fc3d95389...,2


In [120]:
missing = pd.read_csv('alchemy-data/missing.csv', keep_default_na=False)
missing.head()

,blocknum,relay,pubkey
0,16707672,,
1,16707686,,
2,16707713,,
3,16707751,,
4,16707752,blox-maxprofit,0xafc9274fe595e8cff421ab9e73b031f0dff707ea1852...


In [169]:
pairs = missing.groupby(['relay', 'pubkey']).size().reset_index(name='missing')
pairs = pairs.sort_values('missing', ascending=False).reset_index(drop=True)
pairs['connected'] = pairs['pubkey'].isin(ourpks['pubkey'].values)
pairs = pd.merge(pairs, ourpks[['pubkey','description']], on='pubkey', how='left').fillna('')
pairs['pubkey'] = pairs['pubkey'].apply(lambda x: x[:9])
pairs

,relay,pubkey,missing,connected,description
0,,,170,False,
1,blox-maxprofit,0xafc9274,95,False,
2,blocknative,0x8000008,83,False,
3,blox-maxprofit,0x94aa4ee,51,True,bloxroute
4,blox-ethical,0xb086acd,26,False,
5,blox-ethical,0x82801ab,15,False,
6,eden,0x91970c2,9,False,
7,blox-maxprofit,0x976e63c,9,True,bloxroute
8,flashbots,0xb8fceec,6,False,
9,manifold,0x8d01871,6,False,
